<a href="https://colab.research.google.com/github/blue0620/aozorasummary/blob/main/plamo_infer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Plamo-2-2Bを利用した要約モデルの推論チュートリアル
有料版Google Colaboratoryのみで動作します。ランタイムのタイプをL8に設定してください。

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install torch==2.4.0 torchvision==0.19.0 torchaudio==2.4.0 --index-url https://download.pytorch.org/whl/cu121
!pip install mamba_ssm==2.2.2 causal_conv1d==1.4.0
!pip install trl==0.12.2 bitsandbytes datasets

Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 799.1/799.1 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 105.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 94.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 56.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 95.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [14]:
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig
import pandas as pd
from datasets import load_dataset

checkpoint_dir="/content/drive/MyDrive/outputs-plamo2/checkpoint-486"
config = PeftConfig.from_pretrained(checkpoint_dir)
max_seq_length=6000

tokenizer = transformers.AutoTokenizer.from_pretrained(
    "pfnet/plamo-2-1b", model_max_length=max_seq_length,
    padding_side="right", use_fast=False,trust_remote_code=True)
tokenizer.pad_token = tokenizer.unk_token
model = transformers.AutoModelForCausalLM.from_pretrained(
        "pfnet/plamo-2-1b", torch_dtype=torch.bfloat16, device_map="cuda:0",trust_remote_code=True)
model = PeftModel.from_pretrained(model, checkpoint_dir,trust_remote_code=True)
#model.to("cuda:0")
model.generation_config.pad_token_id = tokenizer.pad_token_id
ds = load_dataset('globis-university/aozorabunko-clean')
ds = ds.filter(lambda row: row['meta']['文字遣い種別'] == '新字新仮名')
ds = ds.filter(lambda row: row['meta']['姓'] == '横光')
df = pd.DataFrame(ds["train"])
with torch.no_grad():
    for index,row in df.iterrows():
        question=row["text"][:10000]
        text = f"### 本文: {question}\n ### 要約:"
        qlen=len(text)
        input_ids = tokenizer(text, return_tensors="pt").to("cuda:0")#.input_ids
        generated_tokens = model.generate(
            **input_ids,
            max_new_tokens=200,
            do_sample=True,
            top_k=50,
            temperature=0.3,
            top_p=0.8,
            repetition_penalty=1.5,
            early_stopping=True
        )[0]
        generated_text = tokenizer.decode(generated_tokens)
        print(row["meta"]["作品名"],row["meta"]["図書カードURL"])
        print(generated_text.split("### 要約:")[1].split("\n")[0].split("<|plamo:bos|>")[0])

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

赤い着物 https://www.aozora.gr.jp/cards/000168/card903.html
 この物語は、貧しい農家である主人公とその家族、そして彼らを取り巻く人々との交流を描いています。主人公はある雨の日、妹と一緒に外出し、そこで美しい少女と出会います。その後、主人公が帰宅すると雨が降り始め、その夜遅くには、娘や弟たちが風邪を引いてしまいます。一方、主人公の妻は別の女性と結婚し、その女性が出産のために実家に戻ることになりました。その間、娘は祖母に預けられ、母親が不在の間に世話をします。また、別の人物（おそらく義理の父親）が娘と接触を試みますが、失敗します。さらに、娘が病気になり、父親が仕事に出掛けてしまいます。最後に、この話は様々な登場人物が登場しますが、特に注目すべきなのは、盲導犬と煙草屋です。盲導犬は主人と共に行動し、煙草屋はタバコを売って生計を立てます。この物語は、これらの異なる要素が複雑に入り混じることで、より豊かな雰囲気を作り出しています。<|plamo:eos|>
鵜飼 https://www.aozora.gr.jp/cards/000168/card1110.html
 著者は、この詩の中で「何か」に対する考えを述べています。それは、人々が様々な信念を持ち、それが思考や行動に影響を与えるという考え方です。しかし、哲学者はこの考え方を否定します。なぜなら、人々はそれぞれ異なる信仰を持っており、それが彼ら自身を形成しているからです。この違いによって、彼らは互いに対立することもあれば、協力することもあります。さらに、個人は自分の頭の中だけで物事を考えることが多く、他人の意見を聞く機会が少ないため、自分の考えが正しいかどうか確信が持てないこともあります。このような状況では、理性だけでは問題を解決できず、直感に頼ったり、感情に任せて行動したりすることがあります。しかし、この詩人は、このような複雑な問題に対して、単純な答えを求めてしまいます。彼は、「何かを理解するためには、それを観察し、その本質を見極める必要がある」と主張しています。つまり、物事を理解するには、表面的なことだけでなく、深いところまで掘り下げる必要があるということです。そして、そのような探求を通じてのみ、私たちは真実に近づくことができると信じています。<|plamo:eos|

Token indices sequence length is longer than the specified maximum sequence length for this model (6509 > 6000). Running this sequence through the model will result in indexing errors


南北 https://www.aozora.gr.jp/cards/000168/card50759.html
 物語は、ある人物（主人公）が隣人の娘・安次とその夫・春吉との間に起こった出来事に巻き込まれていく過程を描いています。ある日、主人公が近所に住む女性・秋三と出会います。秋三は裕福そうな雰囲気を持ち、その正体を探るために調査を始めます。その結果、秋三は実は貧乏であり、借金返済のために売られた過去があることが判明します。さらに、秋三は母親と共に住むことを拒否しており、代わりに兄夫婦と一緒に暮らしていることが明らかになります。その後、秋三は病気を患っていることを知りますが、そのことを隠蔽するために嘘をつき、周囲に迷惑をかけてしまいます。一方、主人公は友人を通じて秋三と知り合うようになり、次第に親しくなっていきます。秋三は徐々に主人公に惹かれていきますが、その気持ちを抑えきれず、ついに告白してしまいます。しかし、その直後に悲劇が起こります。主人公は風邪をひいてしまい、高熱を出しながらも懸命に耐え続けます。その甲斐もなく、秋三は再び倒れてしまい、そのまま亡くなってしまいます。悲しみに打ちひしがれる主人公でしたが、やがて立ち直り、新たな人生を歩み始めました。この一連の出来事を通して、読者は様々な人間関係や社会情勢について考えることができます。また、友情や家族の大切さを再認識することができます。<|plamo:eos|>
日輪 https://www.aozora.gr.jp/cards/000168/card4252.html
 この文章は、ある人物である長羅とその家族、そして周囲の人々が繰り広げる物語を描いています。主人公・長羅は、かつて皇族であり、現在は巫女として活躍しています。彼は、弟と共に不彌という土地への侵攻を試みますが、失敗してしまいます。その後、長羅は自分の犯した過ちに気づきます。それは、妹に対する愛情よりも、領土拡大を優先してしまったことでした。その結果、姉との関係にも亀裂が生じてしまいます。さらに、長羅は自分が愛する女性・香取に対しても無関心であったことを認めざるを得ません。これらの出来事をきっかけに、長羅は自らの行動を振り返り、自分の生き方を考え直します。また、長羅を取り巻く家族や仲間も、それぞれが抱える問題に直面し、葛藤していきます。彼らは、それ